In [1]:
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration, Qwen2VLForConditionalGeneration
import torch

# MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"
MODEL_ID = "Qwen/Qwen2-VL-2B"
processor = AutoProcessor.from_pretrained(MODEL_ID)
# model = Qwen2_5_VLForConditionalGeneration.from_pretrained(MODEL_ID)
model = Qwen2VLForConditionalGeneration.from_pretrained(MODEL_ID)


# 비전 인코더만 가져오기
vision_encoder = model.model.visual
# vision_encoder = torch.nn.DataParallel(model.model.visual).eval().cuda()

vision_encoder.eval().cuda()


/clifford-data/home/doyoonkim/miniconda3/envs/doyoon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-24 08:37:32.518718: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-24 08:37:32.541583: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750754252.566407 1348225 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750754252.574506 1348

Qwen2VisionTransformerPretrainedModel(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
  )
  (rotary_pos_emb): VisionRotaryEmbedding()
  (blocks): ModuleList(
    (0-31): 32 x Qwen2VLVisionBlock(
      (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (attn): VisionSdpaAttention(
        (qkv): Linear(in_features=1280, out_features=3840, bias=True)
        (proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (mlp): VisionMlp(
        (fc1): Linear(in_features=1280, out_features=5120, bias=True)
        (act): QuickGELUActivation()
        (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      )
    )
  )
  (merger): PatchMerger(
    (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
    (mlp): Sequential(
      (0): Linear(in_features=5120, out_features=5120, bias=True)
      (1): GEL

In [ ]:
torch.ones(64,)

In [2]:
from PIL import Image
import requests
import torch

# 예시 이미지 불러오기
img = Image.open(requests.get("https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg", stream=True).raw)

# 이미지 전처리 (output: pixel_values, grid_thw)
processed = processor.image_processor(images=img, return_tensors="pt")
pixel_values = processed["pixel_values"].cuda()        # shape: (1, 3, D, H, W)
grid_thw     = processed["image_grid_thw"].cuda()      # shape: (1, 3)


In [3]:
pixel_values.shape

torch.Size([14308, 1176])

In [4]:
grid_thw.shape

torch.Size([1, 3])

In [5]:
with torch.no_grad():
    out = vision_encoder(pixel_values, grid_thw)  # shape: (tokens, dim=2048)


In [14]:
with torch.no_grad():
    out = vision_encoder(pixel_values, grid_thw)  # shape: (tokens, dim=2048)


In [15]:
out.shape

torch.Size([3577, 1536])

In [7]:
vision_encoder.config.hidden_size


1536

In [8]:
vision_encoder.config

Qwen2VLVisionConfig {
  "depth": 32,
  "embed_dim": 1280,
  "hidden_act": "quick_gelu",
  "hidden_size": 1536,
  "in_channels": 3,
  "in_chans": 3,
  "initializer_range": 0.02,
  "mlp_ratio": 4,
  "model_type": "qwen2_vl",
  "num_heads": 16,
  "patch_size": 14,
  "spatial_merge_size": 2,
  "spatial_patch_size": 14,
  "temporal_patch_size": 2,
  "torch_dtype": "float32",
  "transformers_version": "4.52.3"
}

In [9]:
import sys
import os
sys.path.append(os.path.abspath("../vlm_forgetting"))

In [10]:
from attentive_pooler import AttentivePooler, AttentiveClassifier

In [11]:
attentive_pooler = AttentiveClassifier(
        embed_dim=vision_encoder.config.hidden_size,
        num_heads=vision_encoder.config.num_heads,
        depth=1,
        num_classes=10,

    ).cuda()
out = out.cuda()
out = out.unsqueeze(0)
out.shape

torch.Size([1, 3577, 1536])

In [12]:
attentive_pooler(out)

/clifford-data/home/doyoonkim/miniconda3/envs/doyoon/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


tensor([[-0.0374, -0.2016, -0.1618, -0.0218,  0.3811,  0.3574,  0.0160,  0.4745,
          0.2671,  0.2352]], device='cuda:0', grad_fn=<AddmmBackward0>)